#  Team6Project1Py




In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from census import Census

# Census API Key
from config import (api_key, gkey)
c = Census(api_key, year=2018)

# gmaps API Key 
gmaps.configure(api_key=gkey)

# define current date string to append to the names of output files...
import datetime  
def _getToday(): return datetime.date.today().strftime("%Y%m%d")   

### Census.gov API calls to get state and county populations in the U.S.

In [22]:
state_abbr_file="Excel_Files/StateAbbreviation.csv"
state_abbr_df=pd.read_csv(state_abbr_file)
state_abbr_df.head()

,State,State Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [2]:
# Get state Populations...
census_state_data = c.acs5.get(("NAME", "B01003_001E"), {'for': 'state:*'})
# Convert to DataFrame
census_state_df = pd.DataFrame(census_state_data)
# Column Reordering
census_state_df = census_state_df.rename(columns={"NAME": "State", "B01003_001E": "Population"})
del census_state_df['state']
Census_State_Population = "%s_%s.%s" % ("Output_Data/Census_State_Population", _getToday() ,"csv")
census_state_df.to_csv(Census_State_Population, encoding='utf-8', index=False)
census_state_df


# Get County Populations...
census_county_data = c.acs5.get(("NAME", "B01003_001E"), {'for': 'county:*'})
# Convert to DataFrame
census_county_df = pd.DataFrame(census_county_data)
# Column Reordering
census_county_df = census_county_df.rename(columns={"NAME": "Name",  "county": "County","B01003_001E": "Population"})                                     
census_temp_df = census_county_df[['County','State']] = census_county_df.Name.str.split(", ",expand=True,)
census_temp_df = census_county_df[['County','Literal']] = census_county_df.County.str.split(" County",expand=True,)
del census_county_df['Name'] 
del census_county_df['state']
del census_county_df['Literal']
Census_County_Population = "%s_%s.%s" % ("Output_Data/Census_County_Population", _getToday() ,"csv")
census_county_df.to_csv(Census_County_Population, encoding='utf-8', index=False)
census_county_df

,Population,County,State
0,47086.0,Washington,Mississippi
1,12028.0,Perry,Mississippi
2,8321.0,Choctaw,Mississippi
3,23480.0,Itawamba,Mississippi
4,10129.0,Carroll,Mississippi
...,...,...,...
3215,17672.0,Clayton,Iowa
3216,20260.0,Buena Vista,Iowa
3217,10674.0,Guthrie,Iowa
3218,9566.0,Humboldt,Iowa


In [26]:
revised_census_df=pd.merge(census_county_df, state_abbr_df, on="State", how="left")
revised_census_df

,Population,County,State,State Abbreviation
0,47086.0,Washington,Mississippi,MS
1,12028.0,Perry,Mississippi,MS
2,8321.0,Choctaw,Mississippi,MS
3,23480.0,Itawamba,Mississippi,MS
4,10129.0,Carroll,Mississippi,MS
...,...,...,...,...
3215,17672.0,Clayton,Iowa,IA
3216,20260.0,Buena Vista,Iowa,IA
3217,10674.0,Guthrie,Iowa,IA
3218,9566.0,Humboldt,Iowa,IA


### COVID Tracking API Calls to get historical and summary data at the State Level
 

In [ ]:

#get state historical data
covid_state_url = "https://covidtracking.com/api/v1/states/mn/daily.json?"
covidAPI_response_df = pd.DataFrame(requests.get(covid_state_url).json())
covid_state_df = pd.DataFrame (
        {'Date' : covidAPI_response_df['date'],
         'State' : covidAPI_response_df['state'],
         'Positive' : covidAPI_response_df['positive'],
         'Negative' : covidAPI_response_df['negative'],
         'Recovered' : covidAPI_response_df['recovered'],
         'Date_Checked' : covidAPI_response_df['dateChecked'],
         'Death' : covidAPI_response_df['death'],
         'Hospitalized' : covidAPI_response_df['hospitalized'],
         'Total_Test_Results' : covidAPI_response_df['totalTestResults'],
         'Death_Increase' : covidAPI_response_df['deathIncrease'],
         'Hospitalized_Increase' : covidAPI_response_df['hospitalizedIncrease'],
         'Negative_Increase' : covidAPI_response_df['negativeIncrease'],
         'Positive_Increase' : covidAPI_response_df['positiveIncrease'],
         'Total_Test_Results_Increase' : covidAPI_response_df['totalTestResultsIncrease']})
COV_Tracking_MN_History = "%s_%s.%s" % ("Output_Data/COV_Tracking_MN_History_data", _getToday() ,"csv")
covid_state_df.to_csv(COV_Tracking_MN_History, encoding='utf-8', index=False)
covid_state_df 

#get state summary data
covid_currstate_url = "https://covidtracking.com/api/v1/states/mn/current.json?"
covidAPI_currresponse_df = pd.DataFrame(requests.get(covid_currstate_url).json(), index=[0])
covid_currstate_df = pd.DataFrame (        
         {'State' : covidAPI_currresponse_df['state'],
         'Positive' : covidAPI_currresponse_df['positive'],
         'Negative' : covidAPI_currresponse_df['negative'],
         'Recovered' : covidAPI_currresponse_df['recovered'],
         'Date_Checked' : covidAPI_currresponse_df['dateChecked'],
         'Death' : covidAPI_currresponse_df['death'],
         'Hospitalized' : covidAPI_currresponse_df['hospitalized'],
         'Total_Test_Results' : covidAPI_currresponse_df['totalTestResults']})
COV_Tracking_MN_Summary = "%s_%s.%s" % ("Output_Data/COV_Tracking_MN_Summary_data", _getToday() ,"csv")
covid_currstate_df.to_csv(COV_Tracking_MN_Summary, encoding='utf-8', index=False)
covid_currstate_df 

### COVID Tracking API Calls to get summary and historical data  at the National Level

In [ ]:
covid_national_url = "https://covidtracking.com/api/us/daily.json?"
covidAPI_natresp_df = pd.DataFrame(requests.get(covid_national_url).json())
covid_national_df = pd.DataFrame (
        {'Date' : covidAPI_natresp_df['date'],
         'States' : covidAPI_natresp_df['states'],
         'Positive' : covidAPI_natresp_df['positive'],
         'Negative' : covidAPI_natresp_df['negative'],
         'Recovered' : covidAPI_natresp_df['recovered'],
         'Date_Checked' : covidAPI_natresp_df['dateChecked'],
         'Death' : covidAPI_natresp_df['death'],
         'Hospitalized' : covidAPI_natresp_df['hospitalized'],
         'Total_Test_Results' : covidAPI_natresp_df['totalTestResults'],
         'Death_Increase' : covidAPI_natresp_df['deathIncrease'],
         'Hospitalized_Increase' : covidAPI_natresp_df['hospitalizedIncrease'],
         'Negative_Increase' : covidAPI_natresp_df['negativeIncrease'],
         'Positive_Increase' : covidAPI_natresp_df['positiveIncrease'],
         'Total_Test_Results_Increase' : covidAPI_natresp_df['totalTestResultsIncrease']})

COV_Tracking_US_History = "%s_%s.%s" % ("Output_Data/COV_Tracking_US_History_data", _getToday() ,"csv")
covid_national_df.to_csv(COV_Tracking_US_History, encoding='utf-8', index=False)
covid_national_df 


#get national summary data
covid_natsum_url = "https://covidtracking.com/api/v1/states/current.json?"
covidAPI_natsum_df = pd.DataFrame(requests.get(covid_natsum_url).json())
covid_natsum_df = pd.DataFrame (        
         {'State' : covidAPI_natsum_df['state'],
         'Positive' : covidAPI_natsum_df['positive'],
         'Negative' : covidAPI_natsum_df['negative'],
         'Recovered' : covidAPI_natsum_df['recovered'],
         'Date_Checked' : covidAPI_natsum_df['dateChecked'],
         'Death' : covidAPI_natsum_df['death'],
         'Hospitalized' : covidAPI_natsum_df['hospitalized'],
         'Total_Test_Results' : covidAPI_natsum_df['totalTestResults']})
         
COV_Tracking_US_Summary = "%s_%s.%s" % ("Output_Data/COV_Tracking_US_Summary_data", _getToday() ,"csv")
covid_natsum_df.to_csv(COV_Tracking_US_Summary, encoding='utf-8', index=False)
covid_natsum_df 

In [ ]:
# Load the JHU data
JHU_data_df = pd.read_csv("JHU-04-19-2020.csv")
JHU_data = "%s_%s.%s" % ("Output_Data/JHU_data", _getToday() ,"csv")
JHU_data_df.to_csv(JHU_data, encoding='utf-8', index=False)
JHU_data_df

In [ ]:
#plt.plot(covid_national_df)
#plt.plot(covid_state_df)

covid_national_df.plot(kind = 'line', x = 'Date', y = 'Death')